In [ ]:
from hydra import compose, initialize
from omegaconf import OmegaConf
from rdkit import Chem
from rdkit.Chem import rdDetermineBonds
from rdkit.Chem.Draw import IPythonConsole
IPythonConsole.ipython_3d = True
import pandas as pd

from strain_relief.cmdline import strain_relief as sr

In [ ]:
! pip install py3Dmol
import py3Dmol

To run the StrainReleif tool you will need your docked poses in a parquet file. The parquet must contain two columns: your molecule's poses as RDKit.Mol binary strings and a unique ID for each pose.

You likely don't have your poses in this format to begin with so below is a handy function that converts sdfs to this format.

In [ ]:
def sdf_to_parquet(sdf_file, parquet_file):
    suppl = Chem.SDMolSupplier(sdf_file, sanitize=False, removeHs=False)
    mols = [mol for mol in suppl if mol is not None]
    
    df = pd.DataFrame([{"mol_bytes": mol.ToBinary(), **mol.GetPropsAsDict()} for mol in mols])
    df = df.reset_index(drop=False, names='id')
    df.to_parquet(parquet_file)

sdf_path = '../data/example_ligboundconf.sdf'
parquet_path = '../data/example_ligboundconf_input.parquet'

sdf_to_parquet(sdf_path, parquet_path)

Now that we have our poses in the correct format, lets create a hydra config for StrainRelief.

In [ ]:
with initialize(version_base="1.1", config_path="../src/strain_relief/hydra_config"):
    cfg = compose(
        config_name="default", 
        overrides=[
            "experiment=mmff94s",
            f"io.input.parquet_path={parquet_path}"
            ]
        )

print(OmegaConf.to_yaml(cfg))

Now we can run the tool! (This should take less than a minute)

(There are more example scripts in `StrainRelief/examples/`. These can be executed from the command line using the `strain-relief` command or replicate them in the cell above with hydra.)

In [ ]:
results = sr(cfg)
results.head()

The `results` dataframe contains all input columns (in this case `id`, `ligand_id`, `some_property` and `mol_bytes`) and all calculated columns:
- `charge`: RDKit's formal charge. StrainRelief only calculates strains of neutral molecules
- `local_min_mol`: the coordinates of the local minimum
- `local_min_e`: the energy of the local minimum (in kcal/mol)
- `global_min_mol`: the coordinates of the global minimum
- `global_min_e`: the energy of the global minimum (in kcal/mol)
- `ligand_strain`: difference between local and global minima
- `passes_strain_filter`: whether `ligand_strain` is lower than the config threshold
- `nconfs_converged`: the number of conformers that convereged when searching for the global minimum 

Lets have a look at the three poses from ligand 3Q4_3QD0_A_370. 

In [ ]:
docked = Chem.Mol(results.mol_bytes[0])
local_min = Chem.Mol(results.local_min_mol[0])
global_min = Chem.Mol(results.global_min_mol[0])

In [ ]:
rdDetermineBonds.DetermineBonds(docked)
rdDetermineBonds.DetermineBonds(local_min)
rdDetermineBonds.DetermineBonds(global_min)

In [ ]:
IPythonConsole.drawMol3D(docked)

In [ ]:
IPythonConsole.drawMol3D(local_min)

In [ ]:
IPythonConsole.drawMol3D(global_min)

The original and local minimum conformers look very similar to the eye. This is because local minimisation has a loose convergence criteria and is simply to clean up any high energy artifacts left my docking. The global minimum is noticably different, with all aromatic rings having relaxed into similar planes.

You may again want to convert your results back into an sdf. You can do this with the function below:

In [ ]:
def set_properties(mol, properties, row, pose):
    mol.SetProp("pose", pose)
    for p in properties:
        mol.SetProp(p, str(row[p]))
    return mol

def dataframe_to_sdf(df, sdf_file):
    properties = [p for p in df.columns if p not in {"mol_bytes", "local_min_mol", "global_min_mol"}]
    sdf = Chem.SDWriter(sdf_file)

    for _, row in df.iterrows():
        mol = set_properties(Chem.Mol(row.mol_bytes), properties, row, "docked")
        sdf.write(mol)

        local_min = set_properties(Chem.Mol(row.local_min_mol), properties, row, "local_min")
        sdf.write(local_min)

        global_min = set_properties(Chem.Mol(row.global_min_mol), properties, row, "global_min")
        sdf.write(global_min)
    
    sdf.close()

dataframe_to_sdf(results, '../data/example_ligboundconf_output.sdf')

Hopefully you now have a good grasp on how to run the StrainRelief tool! I hope you find it as useful as we have.

Ewan